## Italy Public Company Financial Information Crawler
##### Last Update: 2022-08-02
#### Author: Wonho Lim 
#### E-mail: wonholim02@gmail.com
#### Python Version: Python 3.10.4 (ipykernel)
#### Chrome Version: Chrome 103.0.5060.134 (64-bit)
#### Chrome Driver Version: ChromeDriver 103.0.5060.134

#### Crawled Website
Investing.com MAIN: https://www.investing.com /
Investing.com Italy: https://www.investing.com/equities/italy

Description: This is public company financial statement web crawling code for investing.com, especially for companies in Italy. investing.com is an unofficial stock information website for countries around the world. Even though it is an unofficial website, comparably, it has decent amount of information including key financial values. However, the server itself is very unstable that loading error occurs a lot, and data table format/language format is not standardized that a lot of exceptions are made(goes to failList). So one should be aware of that, and it is better to find alternative source for a long term development. Using excel format instead of csv could be a solution, which I have not tried so far. This code can be reused for other countries by editing initial website address and several page setting button XPATH locations - Colombia, Indonesia, Italy, Sinbgapore, and Spain is already written and uploaded. For crawling, as long as the website has not been modified, the code below must be run properly if and only if it is run in an order. Moreover, environment setting must be fixed based on user's computer/server setting and location.

### 1. Importing useful open source librabries - utilized BS4 and Selenium Web Driver for crawling

In [1]:
import bs4
import time
import csv 
import pandas as pd 
from platform import python_version
import requests
import lxml 
import xlrd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By

### 2. Environment
#### - Chromdriver must be located in local/cloud PC folder.
#### - One can download proper version at: chromedriver.chromium.org/downloads
#### - Path should be modified based on local environment

In [12]:
cover = open('Italy_2022_investing_2022-07-05_Chris.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

### 3. Main Crawler
#### - Uncollectable or unavailable data was collected as "" for convenience.
#### - Data Cleansing might be required after crawling sometiems, but it will not be tough as one knows how to use excel.
#### - Some directions and explanations are written as comment below.
#### - The order of append must be depend on the order of column name assigned above.
#### - Lots of try-except function is used to avoid error caused by non-existing pages or information.
#### - One also need to avoid ad and accept cookie banner that randomly appears.
#### - Run the crawler many times to improve the accuracy - Detail is written below main crawler.

In [13]:
# Basic Setting
failList = []
info = []
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/italy')
# Avoid Ad
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[6]/select').click()
time.sleep(2)
driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[6]/select/option[1]').click()
time.sleep(4)

# Get each company's link using href in a tag
text = driver.page_source
time.sleep(2)
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='cross_rate_markets_stocks_1')
time.sleep(4)
all_atag_maintable = maintable.find_all('a')
print(maintable)
company_links = []

# We have to process the collected data again because of their data structure
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)

# Main Data Collection code
for sub in company_links:
    try:
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        driver.get(base_url + sub)
        info = []
        time.sleep(4)
        #이름
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        #국가정보
        info.append("ITA") 
        info.append("Italy") 
        info.append("Italy") 
        info.append("UTC+01:00") 
        info.append("유럽") 
        info.append("2972000000000 USD") 
        info.append("60317116[") 
        info.append("서유럽") 

        #기업코드
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("ITAHBR" + name) 
        info.append(name)
        info.append(name)
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange."
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company."

        #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Italy"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Italy, Europe"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Italy"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Italy, Europe"
                post = "N/A"
            except:
                address1 = "Italy"
                address2 = "Italy, Europe"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #재무정보
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[2]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[2]').text + "M. EUR")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. EUR")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. EUR")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        #산업군
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        #언어
        info.append("ITA")
        info.append("Italian")
        info.append("Italiano")

        #주식시장정보
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("MIL")
        info.append("Milan Stock Exchange")
        info.append("Milan Stock Exchange")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")

        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "EUR")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Italy")
        info.append("Italy, Asia")
        info.append("")
        info.append("")

        #이벤트
        info.append("")

        #화폐
        info.append("EUR")
        info.append("Euro")

        #관리
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-05")

        print(info)
        writing.writerow(info)
    except:
        print("fail")
        failList.append(base_url + sub)
            
print(failList)

C:\Users\user\AppData\Local\Temp\ipykernel_17824\1136492638.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


<table class="genTbl closedTbl crossRatesTbl elpTbl elp25" id="cross_rate_markets_stocks_1" tablesorter=""><thead><tr><th class="flag"> </th><th class="left noWrap elp pointer">Name<span class="headerSortDefault newSiteIconsSprite" sort_default=""></span></th><th>Last</th><th>High</th><th>Low</th><th data-col-caption="Change">Chg.</th><th class="pointer" data-col-caption="Change %">Chg. %<span class="headerSortDefault newSiteIconsSprite" sort_default="" sortablecache=""></span></th><th class="pointer" data-col-caption="Vol">Vol.<span class="headerSortUp newSiteIconsSprite" sort_default="" sortablecache=""></span></th><th class="pointer" data-col-caption="Time">Time<span class="headerSortDefault newSiteIconsSprite" sort_default="" sortablecache=""></span></th><th class="icon"></th></tr></thead><tbody><tr id="pair_6951"><td class="flag"><span class="ceFlags Italy" title="Italy"> </span></td><td class="bold left noWrap elp plusIconTd"><a href="/equities/intesa-sanpaolo" title="Intesa Sanp

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0000072618', 'Intesa Sanpaolo SpA (ISP)', 'Intesa Sanpaolo SpA (ISP)', "Intesa Sanpaolo S.p.A. is an Italy-based banking company. It operates through six business segments: Banca dei Territori, IMI Corporate & Investment Banking, International Subsidiary Banks, Asset Management, Private Banking and Insurance. Banca dei Territori includes the traditional lending and deposit collecting activities in Italy and related financial services. IMI Corporate & Investment Banking covers corporate banking, investment banking and public finance in Italy and abroad. International Subsidiary Banks oversees the Company's commercial operations on international markets through commercial banking subsidiaries and associates. Private Banking provides private and high net worth individuals with specific products and services. Asset Management develops asset management solutions. Insurance oversees the ma

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005239360', 'UniCredit SpA (CRDI)', 'UniCredit SpA (CRDI)', 'UniCredit SpA is an Italy-based holding company engaged in the financial sector. Its division model is based on seven business lines: Commercial Banking Italy, Central and Eastern Europe (CEE), Corporate and Investment Banking (CIB), Commercial Banking Germany, Commercial Banking Austria and Fineco. UniCredit SpA offers banking and financial services for families, individuals and corporate clients, such as accounts, loans, mortgages, investments, wealth management, as well as financial advice and insurance products, among others. Through numerous subsidiaries, the Company operates in the domestic market, Germany, Austria and CEE.', 'UniCredit SpA (CRDI)(English: UniCredit SpA (CRDI)) is a public company that is listed on Milan Stock Exchange.', 'UniCredit SpA (CRDI)(English: UniCredit SpA (CRDI)) is a public company that i

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0000336518', 'Juventus Football Club (JUVE)', 'Juventus Football Club (JUVE)', 'Juventus FC SpA is an Italy based football club. The Company’s core business is participation in national and international competitions and the organization of matches while exploiting sports events, the Juventus brand and the team image including also the licensing of television and media rights, sponsorship, selling of advertising space, licensing and merchandising. The Company is active mainly locally.', 'Juventus Football Club (JUVE)(English: Juventus Football Club (JUVE)) is a public company that is listed on Milan Stock Exchange.', 'Juventus Football Club (JUVE)(English: Juventus Football Club (JUVE)) is a public company that is listed on Milan Stock Exchange.', None, None, '', '', '', '39 0899 99 98 97', '39 0115 11 92 14', '', 'www.juventus.com', '', '', '', '', '', 'via Druento n. 175, Italy', '

...

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005421885', 'Esi Srl (ESI)', 'Esi Srl (ESI)', 'ESI S.p.A. engages in the renewable energy business primarily in Italy. The company’s business covers engineering, procurement, and construction, as well as system integration activities. It focuses on various market segments, such as photovoltaic, off-grid, mini-grid, hybrid, and storage systems, as well as wind power plants. The company was founded in 2018 and is headquartered in Formello, Italy.', 'Esi Srl (ESI)(English: Esi Srl (ESI)) is a public company that is listed on Milan Stock Exchange.', 'Esi Srl (ESI)(English: Esi Srl (ESI)) is a public company that is listed on Milan Stock Exchange.', None, None, '', '', '', '39 06 98 26 80 08', '39 06 98 26 80 08', '', 'www.esi-spa.com', '', '', '', '', '', 'Via Santa Cornelia 5A, 00060, Italy', 'Via Santa Cornelia 5A, 00060, Italy', '', '', '', '61', '', '', 'Listed', 'Angelo Trementozzi

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005253205', 'Italmobiliare SpA (ITMI)', 'Italmobiliare SpA (ITMI)', 'Italmobiliare SpA is an Italy-based holding company engaged in the investment sector. It operates through two main segment: Industrial and services for industry, and Financial and Private equity. The first segment includes production and market of fresh-food packaging; production and distribution of electric energy from renewable sources; supply of management solutions and other operations. The second segment comprises financial investment companies. The Company operates across Europe, mainly, through its subsidiaries.', 'Italmobiliare SpA (ITMI)(English: Italmobiliare SpA (ITMI)) is a public company that is listed on Milan Stock Exchange.', 'Italmobiliare SpA (ITMI)(English: Italmobiliare SpA (ITMI)) is a public company that is listed on Milan Stock Exchange.', None, None, '', '', '', '39 02 290241', '-', '', 'www

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005068249', 'Costamp Group SpA (MOLDC)', 'Costamp Group SpA (MOLDC)', 'Costamp Group SpA, formerly Modelleria Brambilla SpA, is an Italy-based industrial company. It is active in the Automotive and Industrial Applications markets. The Company operates Die-casting, Low pressure and Gravity, Plastic, Die-holders and Simulation segments through its subsidiaries. The Die-casting segment includes aluminum and magnesium die-casting dies for HPDC (High Pressure Die Casting) machines up to 4500 tons, samples and special batches (medium-small size), among others. The Low pressure and Gravity segment provides low pressure and gravity dies, especially for cylinder heads, crankcases, and core boxes for inorganic process, among others. The Plastic segment offers injection-molding tools for thermoplastics and co-injection molds for thermoplastics, among others. The Die-holders segment manufacture

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005330516', 'Next Re SIIQ SpA (NR)', 'Next Re SIIQ SpA (NR)', 'Next Re SIIQ SpA, formerly known as Nova Re Siiq SpA and as Nova Re SpA, is an Italy-based real estate investment company. The Company focuses on accommodating properties, such as hotels, residential healthcare facilities, student and senior housing. It owns and manages a portfolio of commercial and residential properties across the country, mainly located in Milan, Rome and Bari. The Company is active locally and is subsidiary of SORGENTE SGR SpA.', 'Next Re SIIQ SpA (NR)(English: Next Re SIIQ SpA (NR)) is a public company that is listed on Milan Stock Exchange.', 'Next Re SIIQ SpA (NR)(English: Next Re SIIQ SpA (NR)) is a public company that is listed on Milan Stock Exchange.', None, None, '', '', '', '39 06 81 15 80 00', '-', '', 'www.nextresiiq.it', '', '', '', '', '', 'Via Zara, 28, Italy', 'Via Zara, 28, Italy', '0

pass
pass
fail
pass
pass
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
pass
pass
pass
['ITA', 'Italy', 'Italy',

### 4. Dealing with FailList
#### - Because Investing.com causes loading time error due to unstable server status, it is better to run the code, at least with companies in failList, for several times or run with two or more computers. That will increase the sucess rate much higher than running the code once. When running a new code, make sure to change the name of saving file so that you don't replace the original file.
#### - There are many ways to deal with elements failList: retry(if caused by server loading status), ignore(if caused by actually non-existing data), manually add(if only few exist), find why error has occurred(if massive).
#### - Below is both actual and sample code that was used in 2022

In [14]:
print(len(failList))

423


In [15]:
cover = open('Italy_2022_investing_2022-07-05_Chris - 2.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

In [16]:
failList2 = []
info = []
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/italy')
time.sleep(3)
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
time.sleep(1)
    
for sub in failList:
    try:
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        driver.get(sub)
        info = []
        time.sleep(4)
        #이름
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        #국가코드
        info.append("ITA") 
        info.append("Italy") 
        info.append("Italy") 
        info.append("UTC+01:00") 
        info.append("유럽") 
        info.append("2972000000000 USD") 
        info.append("60317116[") 
        info.append("서유럽") 

        #기업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("ITAHBR" + name) 
        info.append(name)
        info.append(name)
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange"
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company."

        #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Italy"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Italy, Europe"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Italy"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Italy, Europe"
                post = "N/A"
            except:
                address1 = "Italy"
                address2 = "Italy, Europe"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #재무정보
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[2]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[2]').text + "M. EUR")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. EUR")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. EUR")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        #산업군
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        #언어
        info.append("ITA")
        info.append("Italian")
        info.append("Italiano")

        #주식시장정보
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("MIL")
        info.append("Milan Stock Exchange")
        info.append("Milan Stock Exchange")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")

        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "EUR")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Italy")
        info.append("Italy, Asia")
        info.append("")
        info.append("")

        #이벤트
        info.append("")

        #화폐
        info.append("EUR")
        info.append("Euro")

        #관리
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-05")

        print(info)
        writing.writerow(info)
    except:
        print("fail")
        failList2.append(sub)
            
print(failList2)

C:\Users\user\AppData\Local\Temp\ipykernel_17824\3438848392.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


pass
pass
pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005084717', 'Eprice SpA (EPRC)', 'Eprice SpA (EPRC)', 'ePRICE SpA, formerly Banzai SpA, is an Italy-based company engaged in the online retail industry. The Company is an e-commerce operator focused mainly on Tech&Appliances in the domestic market. The E-Commerce segment operates through two websites, such as www.eprice.it, which sells household and electronic appliances for houses, among others, and www.saldiprivati.com, which offers clothes, articles for home and wines, among others.', 'Eprice SpA (EPRC)(English: Eprice SpA (EPRC)) is a public company that is listed on Milan Stock Exchange', 'Eprice SpA (EPRC)(English: Eprice SpA (EPRC)) is a public company that is listed on Milan Stock Exchange', None, None, '', '', '', '-', '-', '', 'corporate-en.eprice.it', '', '', '', '', '', 'Via San Marco, 29, Italy', 'Via San Marco, 29, Italy', '20121', '20121', '20121', '151', ''

pass
pass
pass
fail
pass
pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0003796171', 'Poste Italiane SpA (PST)', 'Poste Italiane SpA (PST)', 'Poste Italiane S.p.A. provides postal, logistics, and financial and insurance products and services in Italy. It operates in four segments: Mail, Parcels and Distribution; Payments and Mobile; Financial Services; and Insurance Services. The Mail, Parcels and Distribution segment offers mail and parcel services, as well as engages in the activities of the distribution network. As of December 31, 2021, it operated a network of 12,761 post offices in Italy. The Payments and Mobile segment provides payment, card payment, and mobile telecommunications services, as well as payment management, e-money services, and mobile and fixed-line telephone services; and digital services for the public sector. The Financial Services segment is involved in the collection of various forms of savings deposits; 

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0003027817', 'Iren (IREE)', 'Iren (IREE)', 'Iren SpA is an Italy-based holding company active in the utilities industry. The Company is engaged in the sector of electricity, thermal energy for district heating and gas, and in the management of integrated water services, environmental services and technological services. The Company is responsible for strategic, development, coordination and control activities, while the four Business Units have been entrusted with the coordination and direction of the companies operating in their respective sectors: Networks, which works in the areas of integrated water cycle, gas distribution and electricity distribution; Waste Management, which carries out waste collection, urban hygiene, treatment and disposal activities; Energy, operating in the sectors of electric and thermal energy production, district heating, energy efficiency and technologic

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'NL0015435975', 'Davide Campari Milano SpA (CPRI)', 'Davide Campari Milano SpA (CPRI)', 'Davide Campari Milano NV, incorporated in the Netherlands, is a company active in the branded spirits industry. It owns an extensive and varied product portfolio of over 50 brands, including Aperol, Appleton Estate, Campari, Grand Marnier, SKYY Vodka and Wild Turkey. It operates through four geographical segments: Americas; Southern Europe, Middle East and Africa; Northern, Central and Eastern Europe; Asia-Pacific. The Company is a global distribution reach, trading in over 190 nations around the world with a focus on Europe and the Americas. It has 21 production plants and its own distribution network in 20 countries.', 'Davide Campari Milano SpA (CPRI)(English: Davide Campari Milano SpA (CPRI)) is a public company that is listed on Milan Stock Exchange', 'Davide Campari Milano SpA (CPRI)(English: 

...

pass
pass
fail
pass
pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005338675', 'Sostravel.Com (SOTR)', 'Sostravel.Com (SOTR)', 'SosTravel.com S.P.A. operates sostravel app that offers passenger assistance services. The company provides information and services on flight tracking, airport info, lost baggage concierge, telemedicine, travel insurance, and other services to airport passengers. It also provides various commercial services to airlines and/or airport handlers, which include the digitization of activities relating to the compilation and forwarding of PIR to the lost and found offices of B2B customers; and functional activities for tracing lost baggage. The company was founded in 2017 and is headquartered in Gallarate, Italy. SosTravel.com S.P.A. is a subsidiary RG HOLDING SRL', 'Sostravel.Com (SOTR)(English: Sostravel.Com (SOTR)) is a public company that is listed on Milan Stock Exchange', 'Sostravel.Com (SOTR)(English:

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0003127898', 'Vianini SpA (VINI)', 'Vianini SpA (VINI)', 'Vianini SpA, formerly Vianini Industria SpA, is an Italy-based company principally engaged in the construction materials industry. The company is active in the manufacture and marketing of concrete products. The Company’s portfolio includes railway sleepers, large-diameter pipes for water, poles for power lines and tunnel segments. The Company’s major shareholder is Caltagirone SpA, which is a holding company active in the areas of construction, real estate, finances and publishing. The Company’s subsidiaries include Ind 2004 Srl, which is engaged in the provision of financial services, and Vianini Energia SpA, which is involved in the production and management of renewable energy installations. Vianini Industria SpA is active in the domestic market. It operates through Domus Italia SpA.', 'Vianini SpA (VINI)(English: Vianini 

pass
pass
fail
pass
pass
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass


pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005429227', 'Igeamed SpA (IGEAM)', 'Igeamed SpA (IGEAM)', 'Igeamed S.p.A. provides consulting, engineering, training, and medical services. The company offers EHS consulting services, such as health and safety, environmental, and energy management; engineering services, including process safety, maintenance engineering, safety engineering, and infrastructure engineering; and corporate healthcare services comprising occupational medicine, workplace health promotion, and corporate welfare, as well as consultancy services related to green power projects. It also provides business assurance, product control and development, infrastructure control, and innovation and smart industry solutions; training solutions for risk management and managerial competence comprising EHS Future Academy, Corporate Future Academy, and Business Assurance Academy; and digital software solutions for the safet

pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
['https://www.investing.com/equities/saras-spa', 'https://www.investing.com/equities/terna', 'https://www.investing.com/equities/tenaris', 'https://www.investing.com/equities/a2a-spa', 'https://www.investing.com/equities/brioschi-sviluppo-immobiliare-spa', 'https://www.investing.com/equities/stmicroelectroni?cid=22185', 'https://www.investing.com/equities/iveco-nv', 'https://www.investing.com/equities/hera-spa', 'https://www.investing.com/equities/pirelli---c', 'https://www.investing.com/equities/mediobanca', 'https://www.investing.com/equities/impregilo', 'https://www.investing.com/equities/ternienergia-spa', 'https://www.investing.com/equities/depobank-spa', 'https://www.investing.com/equities/atlantia', 'https://www.investing.com/eq

In [17]:
print(len(failList2))

343


In [18]:
cover = open('Italy_2022_investing_2022-07-05_Chris - 3.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

In [19]:
failList3 = []
info = []
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/italy')
time.sleep(3)
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
time.sleep(1)
    
for sub in failList2:
    try:
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        driver.get(sub)
        info = []
        time.sleep(4)
        #이름
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        #국가코드
        info.append("ITA") 
        info.append("Italy") 
        info.append("Italy") 
        info.append("UTC+01:00") 
        info.append("유럽") 
        info.append("2972000000000 USD") 
        info.append("60317116[") 
        info.append("서유럽") 

        #기업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("ITAHBR" + name) 
        info.append(name)
        info.append(name)
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange"
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company."

        #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Italy"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Italy, Europe"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Italy"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Italy, Europe"
                post = "N/A"
            except:
                address1 = "Italy"
                address2 = "Italy, Europe"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #재무정보
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[2]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[2]').text + "M. EUR")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. EUR")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. EUR")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        #산업군
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        #언어
        info.append("ITA")
        info.append("Italian")
        info.append("Italiano")

        #주식시장정보
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("MIL")
        info.append("Milan Stock Exchange")
        info.append("Milan Stock Exchange")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")

        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "EUR")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Italy")
        info.append("Italy, Asia")
        info.append("")
        info.append("")

        #이벤트
        info.append("")

        #화폐
        info.append("EUR")
        info.append("Euro")

        #관리
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-05")

        print(info)
        writing.writerow(info)
    except:
        print("fail")
        failList3.append(sub)
            
print(failList3)

C:\Users\user\AppData\Local\Temp\ipykernel_17824\1533889150.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


pass
pass
pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0000433307', 'Saras SpA (SRS)', 'Saras SpA (SRS)', "Saras SpA is an Italy-based company active in the energy sector. The Company operates refineries in Europe. The Company's activities are divided into five segments. The Refining segment refines crude oil into petroleum products at Sarroch refinery. The Marketing segment’s activities focus on the wholesale of oil products aimed at smaller-sized customers, off-network customers and oil companies, as well as third-party operators and the Company-owned service stations, supermarkets and resellers. The Power Generation segment, through Sarlux Srl, focuses on the sale of electricity located within industrial complex in Sarroch. The Wind segment, through Sardeolica Srl, encompasses the production of electricity from renewable sources at the Ulassai wind farm. The Other segment includes environmental protection, measurement system

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005366767', 'Nexi SpA (NEXII)', 'Nexi SpA (NEXII)', 'Nexi S.p.A. provides electronic money and payment services to banks, financial and insurance institutions, merchants, businesses, and public administration in Italy. The company offers acquiring services; configuration, activation, and maintenance of POS terminals; fraud prevention; and dispute management services, as well as customer support services. It also provides a range of issuing services, such as issue, supply, and management of private and corporate payment cards; installation and management of ATM’s; clearing services; and digital banking services for the management of current accounts and payments. In addition, the company offers software applications for invoice management and storage, prepaid card reloading, bill payments, and postal payments. Nexi S.p.A. has a strategic agreement with Intesa Sanpaolo S.p.A. to acqui

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005496473', 'Tiscali (TIS)', 'Tiscali (TIS)', 'Tiscali Spa provides telecommunications services in Italy. The company offers broadband; ultra-broadband; fixed wireless internet access; mobile telephony; and value-added services, including security, cloud, and social media services, as well as digital media and online advertising services through the www.tiscali.it portal. It serves 642,600 thousand household customers. The company was founded in 1998 and is headquartered in Cagliari, Italy.', 'Tiscali (TIS)(English: Tiscali (TIS)) is a public company that is listed on Milan Stock Exchange', 'Tiscali (TIS)(English: Tiscali (TIS)) is a public company that is listed on Milan Stock Exchange', None, None, '', '', '', '39 07 046011', '-', '', 'www.tiscali.com', '', '', '', '', '', 'Loc. Sa Illetta SS 195, Km 2,300, Italy', 'Loc. Sa Illetta SS 195, Km 2,300, Italy', '09123', '09123', '0912

pass
pass
fail
pass
pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005421919', 'Agatos SpA (AGAT)', 'Agatos SpA (AGAT)', 'Agatos SpA, formerly TE Wind SpA, is an Italy-based company engaged in renewable energy sector. The Company focuses on the mini-wind sector and operates wind power plants. It is engaged in the production of energy from renewable sources. The Company’s business model provides coverage for complete steps for the realization of mini-wind turbines, including the location of the site for the production of electricity, as well as management, development and construction of wind systems. In addition, Agatos SpA optimizes the processes of energy use, limiting waste, and self-production through the use of low environmental impact technologies such as photovoltaics, LED lighting and micro cogeneration.', 'Agatos SpA (AGAT)(English: Agatos SpA (AGAT)) is a public company that is listed on Milan Stock Exchange', 'Agatos 

...

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005353815', 'Powersoft SpA (PWS)', 'Powersoft SpA (PWS)', 'Powersoft S.p.A. engages in the design, production, and marketing of power amplifiers, loudspeaker components, and software for installed and live sound applications. The company offers install AMPS, touring AMPS, AMP modules, DSP, tranducers, controllers, accessories, and safety and monitoring system, as well as software. Its products have applications in hospitality industry, including theme parks, resorts and hotels, and bars and restaurants; retail industry comprising malls and shopping centers, and shops; houses of worship; corporate buildings, as well as venues, such as stadiums, arts venues and theatres, convention centers, museums, and clubs; fitness and leisure centers; residential venues, such as home cinema and gaming venues; and government and educational venues. The company was founded in 1995 and is headquarter

pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0003683528', 'Openjobmetis SpA Agenzia per il Lavoro (OJM)', 'Openjobmetis SpA Agenzia per il Lavoro (OJM)', 'Openjobmetis S.p.A. operates as an employment agency in Italy. It offers temporary or permanent placement, staff leasing, personnel recruitment and selection, training, and outplacement services, as well as manages employment relationship from the bureaucratic-administrative aspects to training. The company also provides digital head-hunting services; support services for parent with payroll management tasks and training activities; and family assistants for elderly and non-self-sufficient. It offers its services to the agro-industrial, information and communication technology, health, bank and finance, and large-scale retail trade sectors. Openjobmetis S.p.A. was founded in 2011 and is headquartered in Gallarate, Italy.', 'Openjobmetis SpA Agenzia per il Lavoro (OJM)(English: Ope

pass
pass
fail
pass
pass
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0000214293', 'Toscana Aeroporti SpA (TYA)', 'Toscana Aeroporti SpA (TYA)', 'Toscana Aeroporti SpA is an Italy-based company engaged in the management of airports. The Company is active in the operation of Pisa International Airport and Florence Airport. It divides its activities into two sectors: Aviation and Non Aviation. The Aviation sector includes ground handling services for passengers and aircrafts; landing and take-off services; boarding and disembarking of passengers; aircraft parking; passenger safety 

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0004937469', 'Italia Independent (IINT)', 'Italia Independent (IINT)', 'Italia Independent Group SpA is an Italy-based holding company primarily engaged in production of eyewear. Italia Independent Group SpA specializes in the manufacture and distribution of sunglasses and reading glass frames. Additionally, the Company’s products and services include lifestyle products, such as clothes and accessories, as well as communication services. Italia Independent Group SpA trades its products through Italia Independent brand. The Company operates through its subsidiaries and affiliates, including Sound Identity Srl, We Care Srl, Independent Value Card Srl, Independent Ideas Srl, LA SpA, Italia Independent SpA and Italia Independent USA Corp, among others.', 'Italia Independent (IINT)(English: Italia Independent (IINT)) is a public company that is listed on Milan Stock Exchange', 'Italia Ind

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005105868', 'Cover 50 SpA (COV)', 'Cover 50 SpA (COV)', "Cover 50 SpA is an Italy-based company primarily engaged in the production of apparel. The Company specializes in the designing, manufacturing and selling men's, as well as women's trousers. It distributes clothes under the PT Pantaloni Torino brand name. The Company's product lines comprise a variety of material types, patterns, cuttings and dimensions. Its collection includes PT01, PT01 Ghost Project, PT05, as well as PT Bermuda, among others. Furthermore, the Company operates stationary stores, as well as showrooms in a number of countries, such as Italy, Germany and Japan. It operates through PT Pantaloni Torino Corp.", 'Cover 50 SpA (COV)(English: Cover 50 SpA (COV)) is a public company that is listed on Milan Stock Exchange', 'Cover 50 SpA (COV)(English: Cover 50 SpA (COV)) is a public company that is listed on Milan Sto

In [20]:
print(len(failList3))

272


In [21]:
cover = open('Italy_2022_investing_2022-07-05_Chris - 4.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

In [22]:
failList4 = []
info = []
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/italy')
time.sleep(3)
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
time.sleep(1)
    
for sub in failList3:
    try:
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        driver.get(sub)
        info = []
        time.sleep(4)
        #이름
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        #국가코드
        info.append("ITA") 
        info.append("Italy") 
        info.append("Italy") 
        info.append("UTC+01:00") 
        info.append("유럽") 
        info.append("2972000000000 USD") 
        info.append("60317116[") 
        info.append("서유럽") 

        #기업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("ITAHBR" + name) 
        info.append(name)
        info.append(name)
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange"
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company."

        #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Italy"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Italy, Europe"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Italy"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Italy, Europe"
                post = "N/A"
            except:
                address1 = "Italy"
                address2 = "Italy, Europe"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #재무정보
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[2]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[2]').text + "M. EUR")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. EUR")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. EUR")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        #산업군
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        #언어
        info.append("ITA")
        info.append("Italian")
        info.append("Italiano")

        #주식시장정보
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("MIL")
        info.append("Milan Stock Exchange")
        info.append("Milan Stock Exchange")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")

        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "EUR")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Italy")
        info.append("Italy, Asia")
        info.append("")
        info.append("")

        #이벤트
        info.append("")

        #화폐
        info.append("EUR")
        info.append("Euro")

        #관리
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-05")

        print(info)
        writing.writerow(info)
    except:
        print("fail")
        failList4.append(sub)
            
print(failList4)

C:\Users\user\AppData\Local\Temp\ipykernel_17824\3806554638.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


pass
pass
pass
pass
pass
fail
pass
fail
pass
fail
pass
fail
pass
fail
pass
pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005278236', 'Pirelli & C SPA (PIRC)', 'Pirelli & C SPA (PIRC)', 'Pirelli & C SpA is an Italy-based company. It is a tire player. The Company is the global player focused on the consumer tire market, which includes tiers for cars, motorcycles and bicycles. The tire activity consists of two main segments: the Consumer segment focuses on tires for consumer cars, motor vehicles, sports utility vehicles (SUVs), light commercial vehicles and motorcycles; and the Industrial segment, manufacturing tires for buses, heavy trucks and agricultural machinery. Its products are used by car maker partners such as Bentley, Aston Martin and Porsche, among others. Through its subsidiaries Pirelli & C. Eco Technology SpA, Pirelli & C. Ambiente SpA and PZero Srl, it is active in emissions control technologies, renewable energy so

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005340051', 'Sciuker Frames SpA (SCK)', 'Sciuker Frames SpA (SCK)', 'Sciuker Frames S.p.A. designs and manufactures windows in wood-aluminum and wood-structural glass in Italy. The company operates through Isik, Stratek, Skill, Shutter, Offline, and Exo segments. It offers windows and fixtures. The company was formerly known as System S.r.l. and changed its name to Sciuker Frames S.p.A. in June 2018. The company was founded in 1996 and is headquartered in Contrada, Italy. Sciuker Frames S.p.A operates as a subsidiary of H. Arm S.R.L.', 'Sciuker Frames SpA (SCK)(English: Sciuker Frames SpA (SCK)) is a public company that is listed on Milan Stock Exchange', 'Sciuker Frames SpA (SCK)(English: Sciuker Frames SpA (SCK)) is a public company that is listed on Milan Stock Exchange', None, None, '', '', '', '39 0825 749 84', '39 0825 75 67 91', '', 'www.sciuker.it', '', '', '', '', '', 'Area

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005426215', 'Convergenze  (CVG)', 'Convergenze  (CVG)', 'Convergenze S.p.A. Società Benefit provides nationwide Internet services, voice, energy, and natural gas services. The company offers fiber optic network and WiFi; VoIP and voice services, and marketing services; and cloud data storage, fax, and phone services. It also provides electricity supply service for the private, residential, and business customers; charging points for electric cars; and natural gas supply services. Convergenze S.p.A. Società Benefit was founded in 2005 and is headquartered in Salerno, Italy.', 'Convergenze  (CVG)(English: Convergenze  (CVG)) is a public company that is listed on Milan Stock Exchange', 'Convergenze  (CVG)(English: Convergenze  (CVG)) is a public company that is listed on Milan Stock Exchange', None, None, '', '', '', '39 0828 1962102', '39 0828 1962100', '', 'convergenze.it', '', '', '

pass
pass
fail
pass
pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005497018', 'Neosperience SpA (NSP)', 'Neosperience SpA (NSP)', 'Neosperience S.p.A. develops cloud based software platform. Its platform enables users to understand, engage, and grow customers. The company is based in Milan, Italy.', 'Neosperience SpA (NSP)(English: Neosperience SpA (NSP)) is a public company that is listed on Milan Stock Exchange', 'Neosperience SpA (NSP)(English: Neosperience SpA (NSP)) is a public company that is listed on Milan Stock Exchange', None, None, '', '', '', '39 02 36 75 57 06', '-', '', 'www.neosperience.com', '', '', '', '', '', 'Via Privata Decemviri 20, Italy', 'Via Privata Decemviri 20, Italy', '20137', '20137', '20137', '158', '', '', 'Listed', 'Matteo Linotto', '', 'Director', 'Director', 'Executive Board', '39 02 36 75 57 06', '-', '', '39 02 36 75 57 06', '2021', '21.79M. EUR', '', '47.94M. EUR', '10.61M. EUR', '', '24.66M

...

pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'FR0000073272', 'Safran SA (SAF)', 'Safran SA (SAF)', 'Safran SA is a France-based high-technology company that carries out research, design, development, testing, manufacturing, sales, maintenance, and support operations for its high-technology activities. The Company’s segments are: Aerospace Propulsion, Aircraft Equipment and Aircraft Interiors. The Aerospace Propulsion segment designs, develops, produces, and markets, propulsion and mechanical power transmission systems for commercial aircraft, military transport, training and combat aircraft, civil and military helicopters, satellites, and drones. It also includes maintenance, repair, and overhaul (MRO) activities and the sale of spare parts. The Aircraft Equipment segment operates in five main sectors: Landing and braking systems, Engine systems and equipment, Electrical 

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'GRS802003004', 'Neurosoft Software Production (NRST)', 'Neurosoft Software Production (NRST)', 'Neurosoft Software Production SA is a Greece-based company, which specializes in the design, development, customization and maintenance of integrated software systems. It offers integrated software systems in the following business areas: Factoring and Financials, Sports Betting and Gaming Analytics, Information and Communication Technology (ICT), as well as Management Internal Control Toolset (MICT) and Cascading Style Sheets (CSS). Furthermore, the Company provides information technology services in both the Greek and international markets. Its sports betting products include BOLT, Qualytor, Alkemi, Boom and Betscape. Its receivables finance products include Proxima, Tesla, FoLs and iSynergy. Its cyber security products include Illicium.', 'Neurosoft Software Production (NRST)(English: Neu

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005246142', 'Technical Publications Service SpA (TPSG)', 'Technical Publications Service SpA (TPSG)', 'Technical Publications Service S.p.A. provides technical and engineering services for the aeronautical industry in Italy. It also provides support services to the customers in the design of complex, mechanical, structural, electrical, and avionics systems; and technical publishing and training services. The company was founded in 1964 and is based in Gallarate, Italy.', 'Technical Publications Service SpA (TPSG)(English: Technical Publications Service SpA (TPSG)) is a public company that is listed on Milan Stock Exchange', 'Technical Publications Service SpA (TPSG)(English: Technical Publications Service SpA (TPSG)) is a public company that is listed on Milan Stock Exchange', None, None, '', '', '', '39 03 31 79 70 10', '39 03 31 77 11 07', '', 'www.tps-group.it', '', '', '', '', '

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005337073', 'Longino & Cardenal SPA (LON)', 'Longino & Cardenal SPA (LON)', 'Longino & Cardenal S.p.A. supplies catering services to hotels and restaurants in Italy and internationally. It offers seafood products; plancton Marino; meats and eggs; hams and salamis; cheeses and other dairy products; flowers, sprouts, and aromatic herbs; truffles and mushrooms; breads, pastas, rice, and pulses; salts, spices, and condiments; pastry products; ethnic products and accessories; and champagnes and vodkas. The company was founded in 1988 and is based in Pogliano Milanese, Italy.', 'Longino & Cardenal SPA (LON)(English: Longino & Cardenal SPA (LON)) is a public company that is listed on Milan Stock Exchange', 'Longino & Cardenal SPA (LON)(English: Longino & Cardenal SPA (LON)) is a public company that is listed on Milan Stock Exchange', None, None, '', '', '', '39 029 396851', '-', '', 'www.l

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005394413', 'Doxee Spa (DOXE)', 'Doxee Spa (DOXE)', 'Doxee S.p.A. develops cloud technologies in Italy, the United States, and the Czech Republic. It offers Doxee Document Experience (dX), a customer communications management product for managing business processes related to data transformation and document production, distribution, and archiving; and Doxee Paperless Experience (pX) for dematerialization and digital transformation of documents, such as electronic invoicing, digital storage, and electronic ordering products. The company also provides interactive experience products, such as Doxee Pvideo and Doxee Pweb for digital customer experience. It serves utilities, telecommunications and media, banking, and insurance industries, as well as government institutions. The company was formerly known as Ebilling S.p.A. and changed its name to Doxee S.p.A. in 2012. The company was fo

In [23]:
cover = open('Italy_2022_investing_2022-07-05_Chris - 5.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

In [24]:
failList5 = []
info = []
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/italy')
time.sleep(3)
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
time.sleep(1)
    
for sub in failList4:
    try:
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        driver.get(sub)
        info = []
        time.sleep(4)
        #이름
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        #국가코드
        info.append("ITA") 
        info.append("Italy") 
        info.append("Italy") 
        info.append("UTC+01:00") 
        info.append("유럽") 
        info.append("2972000000000 USD") 
        info.append("60317116[") 
        info.append("서유럽") 

        #기업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("ITAHBR" + name) 
        info.append(name)
        info.append(name)
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange"
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company."

        #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Italy"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Italy, Europe"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Italy"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Italy, Europe"
                post = "N/A"
            except:
                address1 = "Italy"
                address2 = "Italy, Europe"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #재무정보
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[2]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[2]').text + "M. EUR")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. EUR")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. EUR")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        #산업군
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        #언어
        info.append("ITA")
        info.append("Italian")
        info.append("Italiano")

        #주식시장정보
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("MIL")
        info.append("Milan Stock Exchange")
        info.append("Milan Stock Exchange")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")

        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "EUR")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Italy")
        info.append("Italy, Asia")
        info.append("")
        info.append("")

        #이벤트
        info.append("")

        #화폐
        info.append("EUR")
        info.append("Euro")

        #관리
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-05")

        print(info)
        writing.writerow(info)
    except:
        print("fail")
        failList5.append(sub)
            
print(failList5)

C:\Users\user\AppData\Local\Temp\ipykernel_17824\1445541712.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


pass
pass
pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0001233417', 'A2A SpA (A2)', 'A2A SpA (A2)', 'A2A SpA is an Italy-based company engaged in the electric utility sector. The Company operates through five segments. The Energy segment covers electricity generation through hydroelectric and thermoelectric plants, energy management and sale of electricity and gas. The Heat and Services segment facilitates the management of heating plants, as well as distribution of heat in the cities of Milan, Brescia, Bergamo and other municipalities, production and sale of electricity, and maintenance activities at the cogeneration plants. The Environment segment includes waste disposal, street cleaning and waster recovery, among others. The Networks segment encompasses transmission, distribution of electricity, transportation, distribution of natural gas, and water distribution, among others. The Corporate and other services segment include

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0001250932', 'Hera (HRA)', 'Hera (HRA)', 'Hera SpA is an Italy-based company engaged in the energy, environmental and water sectors. The Company’s activities are divided into five business segments: Gas, Electricity, Integrated Water Cycle, Environment and Other Services. The Gas segment includes the distribution and sale of methane gas and Liquefied Petroleum Gas (LPG) services, district heating, as well as heat management. The Electricity segment encompasses electricity production, distribution and sales services. The Integrated Water Cycle segment includes mains water, purification and sewerage services. The Environment segment consists of waste collection, treatment and disposal services. The Other Services segment focuses on public lighting, telecommunications and other minor services. It operates thought Gran Sasso and Blu Ranton.', 'Hera (HRA)(English: Hera (HRA)) is a public 

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005359192', 'illimity Bank SpA (ILTY)', 'illimity Bank SpA (ILTY)', "illimity Bank S.p.A. provides private banking, and investment and trading services in Italy. The company operates through Growth Credit, Distressed Credit, and Direct Banking divisions. The Growth Credit division is involved in the financing of the supply chains of the operators of Italian chains and industrial districts through the activity of recourse and non-recourse purchasing of customers' trade receivables; and financing to businesses; and financing solutions for acquisition activities. It also engages in the purchase of loans that are classified as unlikely-to-pay. The Distressed Credit division is involved in the purchase of secured and unsecured corporate distressed loans on primary and secondary markets; management of corporate non-performing loans portfolios and underlying assets; and provision of financ

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005117848', 'Class Editori SpA (CLED)', 'Class Editori SpA (CLED)', 'Class Editori SpA is an Italy-based company primarily engaged in the publishing sector. The Company business structure is divided into five areas: Newspapers, Periodicals, Electronic Publishing, Professional Services, Television and Radio Channels. The Newspapers area is mainly active in the publishing of Milano Finanza, which is a national financial newspaper. The Periodicals area includes such magazines like Gentleman, Patrimoni, Magazine for Fashion, Capital and Assinews, among others. The Electronic Publishing sector is engaged in the provision of data, information and financial news; trading online products and television (TV) channels dedicated to banks and companies. Through the Professional Services area, the Company is involved in the organization of conferences and shows, sale of advertising space, prepar

...

pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005329815', 'Somec SpA (SOME)', 'Somec SpA (SOME)', 'Somec S.p.A. engineers, designs, and manufactures glass envelopes and façades, special architectural projects, public areas interiors, and professional cooking equipment in Italy and internationally. The company operates in two segments, Seascape and Landscape. It offers doors and windows, balcony balustrades and partitions, prefabricated balconies, continuous glass walls, skylights and winter gardens, and wheelhouse and special glazing products, as well as fire resistant products. The company also provides marine refitting services; and designs, produces, and commissions catering solutions for cruise ships. In addition, it offers marine interiors outfitting services, including newbuilding and refit of cruise liners, private yachts, and military vessels; and engineer solutions for building f

pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'FR0000121485', 'Kering SA (PRTP)', 'Kering SA (PRTP)', 'Kering SA is a France-based luxury group. The Group manages the development of a series of renowned Houses in Fashion, Leather Goods and Jewelry: Gucci, Saint Laurent, Bottega Veneta, Balenciaga, Alexander McQueen, Brioni, Boucheron, Pomellato, DoDo, Qeelin, as well as Kering Eyewear. It manufactures and sell, mostly through managed retail stores, an array of products, including leather goods, apparel, accessories, footwear, jewelry among others, for man, woman and child. The Group is active globally.', 'Kering SA (PRTP)(English: Kering SA (PRTP)) is a public company that is listed on Milan Stock Exchange', 'Kering SA (PRTP)(English: Kering SA (PRTP)) is a public company that is listed on Milan Stock Exchange', None, None, '', '', '', '33 1 45 64 61 00', '-', '', 'www.kering.com', '', '', ''

pass
pass
fail
pass
pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005245508', 'Indel B SPA (INDB)', 'Indel B SPA (INDB)', 'Indel B SpA is an Italy-based company engaged in the electrical components and equipment industry. Indel B SpA operates through two business lines, such as the Horeca sector, which supplies international hotel chains, minibars and also accessories for hotel rooms, such electronic safes and locks, light-emitting diode (LED) TVs, hairdryers, LED lighting and wine cellars, as well as the Mobile Refrigeration market, that produces refrigerators for trucks, such as the European trucks and Pullman builders, such as Scania, Iveco, Daf, Renault, Volvo, Kenworth, Peterbilt, Irizar, Irisbus, Karosa. The Company focuses on the commercial exploitation of the invention of the semi-hermetic compressor for generating cold temperatures in small spaces.', 'Indel B SPA (INDB)(English: Indel B SPA (INDB)) is a public company 

pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'DE0008430026', 'Muench. Rueckvers. VNA O.N. (MUV2)', 'Muench. Rueckvers. VNA O.N. (MUV2)', 'Münchener Rückversicherungs-Gesellschaft Aktiengesellschaft in München engages in the insurance and reinsurance businesses worldwide. The company operates through five segments: Life and Health Reinsurance; Property-Casualty Reinsurance; ERGO Life and Health Germany; ERGO Property-Casualty Germany; and ERGO International. It offers life and health reinsurance solutions, such as financial market risks, data analytics, claims handling and underwriting, medical research, health market, capital management, and digital underwriting and analytics, as well as MIRA digital suite; and property and casualty reinsurance solutions, including agricultural risk, data analytics, infrastructure risk profiler, remote inspection, retroactive reinsurance, insurance linked se

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005388217', 'Fos SpA (FOS)', 'Fos SpA (FOS)', 'Fos S.p.A. provides technical consultancy services in Italy. The company engages in design and implementation of software solutions for health/medical and transport and shipping industries, banks and insurance companies, central and local public administrations; environment development; design software development; and software configuration management. It is also involved in design and implementation network operations centers and access solutions to business applications; design, installation, and management of local network and datacenter architectures, and corporate VPNs and application portals; design and management of geographic networks and dynamic routing protocols; provision of cyber security solutions comprising design and operational management of security architectures; definition and implementation of IS policies for sensit

In [25]:
print(len(failList5))

171


In [26]:
cover = open('Italy_2022_investing_2022-07-05_Chris - 6.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

In [27]:
failList6 = []
info = []
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/italy')
time.sleep(3)
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
time.sleep(1)
    
for sub in failList5:
    try:
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        driver.get(sub)
        info = []
        time.sleep(4)
        #이름
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        #국가코드
        info.append("ITA") 
        info.append("Italy") 
        info.append("Italy") 
        info.append("UTC+01:00") 
        info.append("유럽") 
        info.append("2972000000000 USD") 
        info.append("60317116[") 
        info.append("서유럽") 

        #기업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("ITAHBR" + name) 
        info.append(name)
        info.append(name)
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange"
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company."

        #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Italy"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Italy, Europe"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Italy"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Italy, Europe"
                post = "N/A"
            except:
                address1 = "Italy"
                address2 = "Italy, Europe"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #재무정보
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[2]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. EUR")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. EUR")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[2]').text + "M. EUR")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. EUR")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. EUR")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. EUR")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. EUR")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        #산업군
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        #언어
        info.append("ITA")
        info.append("Italian")
        info.append("Italiano")

        #주식시장정보
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("MIL")
        info.append("Milan Stock Exchange")
        info.append("Milan Stock Exchange")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")

        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "EUR")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Italy")
        info.append("Italy, Asia")
        info.append("")
        info.append("")

        #이벤트
        info.append("")

        #화폐
        info.append("EUR")
        info.append("Euro")

        #관리
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-05")

        print(info)
        writing.writerow(info)
    except:
        print("fail")
        failList6.append(sub)
            
print(failList6)

C:\Users\user\AppData\Local\Temp\ipykernel_17824\1674282372.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


pass
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005239881', 'UniEuro SpA (UNIR)', 'UniEuro SpA (UNIR)', 'Unieuro S.p.A. distributes consumer electronics and appliances in Italy and internationally. It offers telephone systems, computers, photos, large and small appliances, consumer electronics, and media storage products; and other products, including consoles, video games, and bicycles, as well as ancillary services, such as delivery and installation, extended warranties, and consumer financing services. The company provides its products through online, indirect, business to business, and travel distribution channels. As of February 28, 2022, it operated 271 direct sales outlets. The company was formerly known as S.G.M. Distribuzione S.r.l. and changed its name to Unieuro S.p.A. in December 2016. Unie

pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005340655', 'Renergetica (RENER)', 'Renergetica (RENER)', "Renergetica S.p.A., a smart energy company, provides a suite of development, engineering, and asset management services for renewable energy plants and energy storage solutions worldwide. The company’s development services include market analysis and feasibility, project feasibility, authorization, and project sale. It also provides engineering services, such as due diligence, design, and owner engineering; and engages in the engineering and project management of solar, wind, and hydro power plants of third parties, as well as hybrid systems. In addition, the company engages in the provision of technical and administrative management services for the solar and wind plants comprising plant control and monitoring, maintenance supervision, commercial and insurance management, and budgeting and reporting services. Renergetica S.

pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'IT0005332595', 'Grifal SpA (GRALA)', 'Grifal SpA (GRALA)', 'Grifal S.p.A. provides packaging solutions in Italy and rest of Europe. It offers corrugated cardboard packaging solutions, such as flexible inserts, blocking and bracing, separators, protection and void fill, cushioning bottoms, custom-cut inserts, and edge protection solutions, as well as cushioning boxes; cushion paper packaging solutions for protection, separation, blocking, wrapping, and filling applications; rolls or sheets; and protective packaging solutions under the cArtù, Mondaplen, cushionPaper, and inspiropack brands. The company was founded in 1969 and is based in Cologno Al Serio, Italy. Grifal S.p.A. is a subsidiary of G-Quattronove S.R.L.', 'Grifal SpA (GRALA)(English: Grifal SpA (GRALA)) is a public company that

pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'ES0148396007', 'Industria de Diseno Textil SA (ITX)', 'Industria de Diseno Textil SA (ITX)', 'Industria de Diseño Textil, S.A. engages in the retail and online distribution of clothing, footwear, accessories, and household textile products through various commercial concepts. Its retail concepts include Zara, Pull & Bear, Massimo Dutti, Bershka, Stradivarius, Oysho, Zara Home, and Uterqüe. As of July 13, 2021, the company operated 6,829 stores in 96 markets; and online stores in 216 markets. It is also involved in textile manufacturing, design, financial services, real estate, logistics, insurance, and combined heat and power plant, and construction businesses. The company operates in Spai

pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
pass
pass
pass
fail
['https://www.investing.com/equities/seri-industrial-spa-wnt', 'https://www.investing.com/equities/solid-world', 'https://www.investing.com/equities/defence-tech-holding', 'https://www.investing.com/equities/laboratorio-farmaceutico-erfo', 'https://www.investing.com/equities/industrie-de-nora-sp', 'https://www.inv

In [2]:
cover = open('Italy_2022_investing_2022-07-05_Chris - 7.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

In [4]:
failList6 = ['https://www.investing.com/equities/seri-industrial-spa-wnt', 'https://www.investing.com/equities/solid-world', 'https://www.investing.com/equities/defence-tech-holding', 'https://www.investing.com/equities/laboratorio-farmaceutico-erfo', 'https://www.investing.com/equities/industrie-de-nora-sp', 'https://www.investing.com/equities/air-france---klm?cid=1122332', 'https://www.investing.com/equities/unieuro-spa', 'https://www.investing.com/equities/sciuker-frames', 'https://www.investing.com/equities/convergenze-wnt', 'https://www.investing.com/equities/expert-system', 'https://www.investing.com/equities/meglioquesto-wnt', 'https://www.investing.com/equities/convergenze', 'https://www.investing.com/equities/faurecia?cid=1172324', 'https://www.investing.com/equities/dt-lufthansa?cid=985888', 'https://www.investing.com/equities/high-quality-food', 'https://www.investing.com/equities/farmacosmo', 'https://www.investing.com/equities/thyssen-krupp?cid=990000', 'https://www.investing.com/equities/rocket-sharing', 'https://www.investing.com/equities/sourcesense-spa-wnt', 'https://www.investing.com/equities/altea-green-power', 'https://www.investing.com/equities/soluzione-tasse', 'https://www.investing.com/equities/redelfi', 'https://www.investing.com/equities/tesla-motors?cid=1082071', 'https://www.investing.com/equities/deutsche-bank?cid=29440', 'https://www.investing.com/equities/civitanavi-systems', 'https://www.investing.com/equities/finanza-tech-sb', 'https://www.investing.com/equities/amazon-com-inc?cid=1055958', 'https://www.investing.com/equities/fos-wnt', 'https://www.investing.com/equities/sababa-security', 'https://www.investing.com/equities/renault?cid=29469', 'https://www.investing.com/equities/qf-mediolanum-re-b', 'https://www.investing.com/equities/ulisse-biomed-spa', 'https://www.investing.com/equities/datrix', 'https://www.investing.com/equities/intermonte-partners-sim', 'https://www.investing.com/equities/estrima', 'https://www.investing.com/equities/piquadro-spa', 'https://www.investing.com/equities/spindox-spa', 'https://www.investing.com/equities/newlat-food', 'https://www.investing.com/equities/ilpra', 'https://www.investing.com/equities/destination-italia', 'https://www.investing.com/equities/gdf-suez?cid=29448', 'https://www.investing.com/equities/telefonica?cid=29476', 'https://www.investing.com/equities/apple-computer-inc?cid=1095913', 'https://www.investing.com/equities/nokia-finland?cid=29485', 'https://www.investing.com/equities/grifal', 'https://www.investing.com/equities/volkswagen-vz?cid=985972', 'https://www.investing.com/equities/cofle-wnt', 'https://www.investing.com/equities/commerzbank-ag?cid=985887', 'https://www.investing.com/equities/societa-editoriale-il', 'https://www.investing.com/equities/bayer-ag?cid=29437', 'https://www.investing.com/equities/bb-biotech-ag-exch?cid=29587', 'https://www.investing.com/equities/e.on?cid=29584', 'https://www.investing.com/equities/destination-italia-wnt', 'https://www.investing.com/equities/banco-santander?cid=29480', 'https://www.investing.com/equities/intel-corp?cid=1122336', 'https://www.investing.com/equities/bay-mot-werke?cid=29486', 'https://www.investing.com/equities/bellini-nautica', 'https://www.investing.com/equities/facebook-inc?cid=1122333', 'https://www.investing.com/equities/credit-agricole?cid=29457', 'https://www.investing.com/equities/promotica-spa-wnt', 'https://www.investing.com/equities/sif-italia', 'https://www.investing.com/equities/lindbergh', 'https://www.investing.com/equities/nvidia-corp?cid=1120644', 'https://www.investing.com/equities/basf-ag?cid=29436', 'https://www.investing.com/equities/k-s-ag?cid=986092', 'https://www.investing.com/equities/societe-generale?cid=29460', 'https://www.investing.com/equities/philips-kon?cid=29425', 'https://www.investing.com/equities/generalfinance', 'https://www.investing.com/equities/dba', 'https://www.investing.com/equities/ing-groep?cid=29423', 'https://www.investing.com/equities/ahold-kon?cid=29525', 'https://www.investing.com/equities/id-entity', 'https://www.investing.com/equities/revo-spa-warrants', 'https://www.investing.com/equities/directa-sim', 'https://www.investing.com/equities/ala-spa', 'https://www.investing.com/equities/daimler?cid=29441', 'https://www.investing.com/equities/nusco', 'https://www.investing.com/equities/dt-telekom?cid=29487', 'https://www.investing.com/equities/sanofi-aventis?cid=29463', 'https://www.investing.com/equities/inditex?cid=1161082', 'https://www.investing.com/equities/farmae-srl', 'https://www.investing.com/equities/integrated-system-credit-consulting', 'https://www.investing.com/equities/allianz-ag?cid=29438', 'https://www.investing.com/equities/microsoft-corp?cid=1122337', 'https://www.investing.com/equities/netflix,-inc.?cid=1122358', 'https://www.investing.com/equities/iberdrola?cid=1117821', 'https://www.investing.com/equities/axa?cid=29445', 'https://www.investing.com/equities/france-telecom?cid=29458', 'https://www.investing.com/equities/siemens?cid=29439', 'https://www.investing.com/equities/bifire', 'https://www.investing.com/equities/evonik-industries-ag?cid=992777', 'https://www.investing.com/equities/eads?cid=985965', 'https://www.investing.com/equities/carrefour?cid=29456', 'https://www.investing.com/equities/pro-sieben?cid=987105', 'https://www.investing.com/equities/continental-ag?cid=985966', 'https://www.investing.com/equities/adv-micro-device?cid=1122331', 'https://www.investing.com/equities/infineon-tech?cid=985969', 'https://www.investing.com/equities/fonc-des-regions?cid=1121712', 'https://www.investing.com/equities/essilor-internat?cid=989452', 'https://www.investing.com/equities/total?cid=29468', 'https://www.investing.com/equities/rwe-st-a?cid=29492', 'https://www.investing.com/equities/danone?cid=29446', 'https://www.investing.com/equities/racing-force', 'https://www.investing.com/equities/industrial-stars-of-italy-4-spa', 'https://www.investing.com/equities/medica', 'https://www.investing.com/equities/anheuser-busch-inbev?cid=987102', 'https://www.investing.com/equities/asml-holding?cid=989451', 'https://www.investing.com/equities/heidelbergcement-ag-exch?cid=986090', 'https://www.investing.com/equities/l.v.m.h.?cid=29451', 'https://www.investing.com/equities/google-inc?cid=1159096', 'https://www.investing.com/equities/gilead-sciences-inc?cid=1158951', 'https://www.investing.com/equities/qf-socrate', 'https://www.investing.com/equities/merck-kgaa?cid=985971', 'https://www.investing.com/equities/l-oreal?cid=29453', 'https://www.investing.com/equities/adidas-salomon?cid=985964', 'https://www.investing.com/equities/somec', 'https://www.investing.com/equities/fresenius-medi?cid=987104', 'https://www.investing.com/equities/id-entity-wnt', 'https://www.investing.com/equities/take-off', 'https://www.investing.com/equities/svas-biosana', 'https://www.investing.com/equities/sap-ag?cid=29489', 'https://www.investing.com/equities/schneider-electr?cid=986259', 'https://www.investing.com/equities/cofle', 'https://www.investing.com/equities/puma-se-cfd?cid=1161083', 'https://www.investing.com/equities/star7', 'https://www.investing.com/equities/henkel-hgaa-vz?cid=989453', 'https://www.investing.com/equities/ageas?cid=29586', 'https://www.investing.com/equities/repsol-ypf?cid=1117822', 'https://www.investing.com/equities/vivendi?cid=29465', 'https://www.investing.com/equities/beiersdorf?cid=986087', 'https://www.investing.com/equities/deutsche-annington-immobilien-se?cid=990002', 'https://www.investing.com/equities/muench.-rueck?cid=29488', 'https://www.investing.com/equities/deutsche-post?cid=985967', 'https://www.investing.com/equities/google-inc-c?cid=1122334', 'https://www.investing.com/equities/qf-immobiliare-dinamico', 'https://www.investing.com/equities/amgen-inc?cid=1161072', 'https://www.investing.com/equities/fresenius-ag?cid=986089', 'https://www.investing.com/equities/starbucks-corp?cid=1122360', 'https://www.investing.com/equities/4aim-societa-di-investimento', 'https://www.investing.com/equities/aegon?cid=29424', 'https://www.investing.com/equities/vinci?cid=985889', 'https://www.investing.com/equities/dt-boerse?cid=987103', 'https://www.investing.com/equities/saint-gobain?cid=987106', 'https://www.investing.com/equities/uniball?cid=991178', 'https://www.investing.com/equities/bbva?cid=1161076', 'https://www.investing.com/equities/tripadvisor?cid=1161084', 'https://www.investing.com/equities/amundi-re-europa', 'https://www.investing.com/equities/vontobel-financial-products-gmbh', 'https://www.investing.com/equities/osai-automation-system-spa-wnt', 'https://www.investing.com/equities/esi-spa-wnt', 'https://www.investing.com/equities/tenax-international-spa-wnt', 'https://www.investing.com/equities/vantea-smart-spa-wnt', 'https://www.investing.com/equities/radici-pietro-industries-brands-wnt', 'https://www.investing.com/equities/trendevice-spa-wnt', 'https://www.investing.com/equities/antares-vision-spa-wnt', 'https://www.investing.com/equities/trawell-co-spa-wnt', 'https://www.investing.com/equities/ulisse-biomed-spa-wnt', 'https://www.investing.com/equities/nusco-spa-con-unico-socio-wnt-24', 'https://www.investing.com/equities/cyberoo-wnt', 'https://www.investing.com/equities/industrie-chimiche-forestali-wnt', 'https://www.investing.com/equities/matica-fintec-wnt', 'https://www.investing.com/equities/nice-footwear', 'https://www.investing.com/equities/alfonsino', 'https://www.investing.com/equities/vontobel-financial-products-wnt', 'https://www.investing.com/equities/alfonsino-wnt', 'https://www.investing.com/equities/international-care', 'https://www.investing.com/equities/homizy']
failList7 = []
info = []
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/italy')
time.sleep(3)
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
time.sleep(1)
    
for sub in failList6:
    info = []
    try:
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        driver.get(sub)
        info = []
        time.sleep(4)
        #이름
        try:
            name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        except:
            name = "Fail"
        #국가코드
        info.append("ITA") 
        info.append("Italy") 
        info.append("Italy") 
        info.append("UTC+01:00") 
        info.append("유럽") 
        info.append("2972000000000 USD") 
        info.append("60317116[") 
        info.append("서유럽") 

        #기업
        info.append("ITAHBR" + name) 
        info.append(name)
        info.append(name)
        time.sleep(1)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        description_short = name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange."
        description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company."

        #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
        address1 = "Italy"
        address2 = "Italy, Europe"
        post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        info.append("")
        info.append("")
        info.append("Director")
        info.append("Director")
        info.append("Executive Board")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        info.append("")
        #유동자산금액
        info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        info.append("")
        #유동부채금액
        info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        info.append("")
        #자본총계금액
        info.append("")
        #자본부채총계금액
        info.append("")
        #매출액
        info.append("")
        #매출원가금액
        info.append("")
        #판매비관리비금액
        info.append("")
        #영업이익손실금액
        info.append("")
        #금융수익금액
        info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        info.append("")
        #법인세비용
        info.append("")
        #당기순이익
        info.append("")
        #현금흐름
        #영업
        info.append("")
        #투자
        info.append("")
        #재무
        info.append("")
        #기초기말
        info.append("")
        info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        info.append("Public Company")
        info.append("Public Company")

        info.append("Product and Services")
        info.append("Product and Services")


        #언어
        info.append("ITA")
        info.append("Italian")
        info.append("Italiano")

        #주식시장정보
        info.append("MIL")
        info.append("Milan Stock Exchange")
        info.append("Milan Stock Exchange")
        info.append("")
        #상장일자
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        #거래량
        info.append("")
        #시가총액
        info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Italy")
        info.append("Italy, Asia")
        info.append("")
        info.append("")

        #이벤트
        info.append("")

        #화폐
        info.append("EUR")
        info.append("Euro")

        #관리
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-05")
        print(info)
        writing.writerow(info)
    except:
        failList7.append(sub)
            
print(failList7)

C:\Users\user\AppData\Local\Temp\ipykernel_13180\179735295.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


pass
pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'ITAHBRKREnergy SpA 30 Dec 22 (SERK_t)', 'KREnergy SpA 30 Dec 22 (SERK_t)', 'KREnergy SpA 30 Dec 22 (SERK_t)', 'KREnergy SpA 30 Dec 22 (SERK_t)(English: KREnergy SpA 30 Dec 22 (SERK_t)) is a public company that is listed on Milan Stock Exchange.', 'KREnergy SpA 30 Dec 22 (SERK_t)(English: KREnergy SpA 30 Dec 22 (SERK_t)) is a public company that is listed on Milan Stock Exchange.', 'KREnergy SpA 30 Dec 22 (SERK_t)(English: KREnergy SpA 30 Dec 22 (SERK_t)) is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company.', 'KREnergy SpA 30 Dec 22 (SERK_t)(English: KREnergy SpA 30 Dec 22 (SERK_t)) is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company.', '', '', '', '', '', '', '', '', '', '', '', '', 'Italy', 'Italy'

pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'ITAHBRUniEuro SpA (UNIR)', 'UniEuro SpA (UNIR)', 'UniEuro SpA (UNIR)', 'UniEuro SpA (UNIR)(English: UniEuro SpA (UNIR)) is a public company that is listed on Milan Stock Exchange.', 'UniEuro SpA (UNIR)(English: UniEuro SpA (UNIR)) is a public company that is listed on Milan Stock Exchange.', 'UniEuro SpA (UNIR)(English: UniEuro SpA (UNIR)) is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company.', 'UniEuro SpA (UNIR)(English: UniEuro SpA (UNIR)) is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company.', '', '', '', '', '', '', '', '', '', '', '', '', 'Italy', 'Italy', 'Italy, Europe', 'Italy, Europe', 'N/A', '', '', '', 'Listed', '', '', 'Director', 'Director', 'Executive Board', '', '', '', '', '', '', '', '', 

pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'ITAHBRConvergenze  (CVG)', 'Convergenze  (CVG)', 'Convergenze  (CVG)', 'Convergenze  (CVG)(English: Convergenze  (CVG)) is a public company that is listed on Milan Stock Exchange.', 'Convergenze  (CVG)(English: Convergenze  (CVG)) is a public company that is listed on Milan Stock Exchange.', 'Convergenze  (CVG)(English: Convergenze  (CVG)) is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company.', 'Convergenze  (CVG)(English: Convergenze  (CVG)) is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company.', '', '', '', '', '', '', '', '', '', '', '', '', 'Italy', 'Italy', 'Italy, Europe', 'Italy, Europe', 'N/A', '', '', '', 'Listed', '', '', 'Director', 'Director', 'Executive Board', '', '', '', '', '', '', '', '', 

pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'ITAHBRRocket Sharing Company SpA (RCKT)', 'Rocket Sharing Company SpA (RCKT)', 'Rocket Sharing Company SpA (RCKT)', 'Rocket Sharing Company SpA (RCKT)(English: Rocket Sharing Company SpA (RCKT)) is a public company that is listed on Milan Stock Exchange.', 'Rocket Sharing Company SpA (RCKT)(English: Rocket Sharing Company SpA (RCKT)) is a public company that is listed on Milan Stock Exchange.', 'Rocket Sharing Company SpA (RCKT)(English: Rocket Sharing Company SpA (RCKT)) is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company.', 'Rocket Sharing Company SpA (RCKT)(English: Rocket Sharing Company SpA (RCKT)) is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company.', '', '', '', '', '', '', '', '', '', '', '', '',

...

pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'ITAHBRTripAdvisor Inc (TTRIP)', 'TripAdvisor Inc (TTRIP)', 'TripAdvisor Inc (TTRIP)', 'TripAdvisor Inc (TTRIP)(English: TripAdvisor Inc (TTRIP)) is a public company that is listed on Milan Stock Exchange.', 'TripAdvisor Inc (TTRIP)(English: TripAdvisor Inc (TTRIP)) is a public company that is listed on Milan Stock Exchange.', 'TripAdvisor Inc (TTRIP)(English: TripAdvisor Inc (TTRIP)) is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company.', 'TripAdvisor Inc (TTRIP)(English: TripAdvisor Inc (TTRIP)) is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company.', '', '', '', '', '', '', '', '', '', '', '', '', 'Italy', 'Italy', 'Italy, Europe', 'Italy, Europe', 'N/A', '', '', '', 'Listed', '', '', 'Director', 'Directo

pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'ITAHBRVantea Smart SpA Call 3.08 EUR Vantea Smart SpA Ord 31Oct22 (VNT_t)', 'Vantea Smart SpA Call 3.08 EUR Vantea Smart SpA Ord 31Oct22 (VNT_t)', 'Vantea Smart SpA Call 3.08 EUR Vantea Smart SpA Ord 31Oct22 (VNT_t)', 'Vantea Smart SpA Call 3.08 EUR Vantea Smart SpA Ord 31Oct22 (VNT_t)(English: Vantea Smart SpA Call 3.08 EUR Vantea Smart SpA Ord 31Oct22 (VNT_t)) is a public company that is listed on Milan Stock Exchange.', 'Vantea Smart SpA Call 3.08 EUR Vantea Smart SpA Ord 31Oct22 (VNT_t)(English: Vantea Smart SpA Call 3.08 EUR Vantea Smart SpA Ord 31Oct22 (VNT_t)) is a public company that is listed on Milan Stock Exchange.', 'Vantea Smart SpA Call 3.08 EUR Vantea Smart SpA Ord 31Oct22 (VNT_t)(English: Vantea Smart SpA Call 3.08 EUR Vantea Smart SpA Ord 31Oct22 (VNT_t)) is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are 

pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'ITAHBRUlisse Biomed SpA Warrants 30Oct26 (UBM_t)', 'Ulisse Biomed SpA Warrants 30Oct26 (UBM_t)', 'Ulisse Biomed SpA Warrants 30Oct26 (UBM_t)', 'Ulisse Biomed SpA Warrants 30Oct26 (UBM_t)(English: Ulisse Biomed SpA Warrants 30Oct26 (UBM_t)) is a public company that is listed on Milan Stock Exchange.', 'Ulisse Biomed SpA Warrants 30Oct26 (UBM_t)(English: Ulisse Biomed SpA Warrants 30Oct26 (UBM_t)) is a public company that is listed on Milan Stock Exchange.', 'Ulisse Biomed SpA Warrants 30Oct26 (UBM_t)(English: Ulisse Biomed SpA Warrants 30Oct26 (UBM_t)) is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company.', 'Ulisse Biomed SpA Warrants 30Oct26 (UBM_t)(English: Ulisse Biomed SpA Warrants 30Oct26 (UBM_t)) is a public company that is listed on Milan Stock Exchange. They are providing products/services i

pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'ITAHBRNice Footwear SpA (NFTW)', 'Nice Footwear SpA (NFTW)', 'Nice Footwear SpA (NFTW)', 'Nice Footwear SpA (NFTW)(English: Nice Footwear SpA (NFTW)) is a public company that is listed on Milan Stock Exchange.', 'Nice Footwear SpA (NFTW)(English: Nice Footwear SpA (NFTW)) is a public company that is listed on Milan Stock Exchange.', 'Nice Footwear SpA (NFTW)(English: Nice Footwear SpA (NFTW)) is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company.', 'Nice Footwear SpA (NFTW)(English: Nice Footwear SpA (NFTW)) is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company.', '', '', '', '', '', '', '', '', '', '', '', '', 'Italy', 'Italy', 'Italy, Europe', 'Italy, Europe', 'N/A', '', '', '', 'Listed', '', '', 'Director

pass
pass
['ITA', 'Italy', 'Italy', 'UTC+01:00', '유럽', '2972000000000 USD', '60317116[', '서유럽', 'ITAHBRHomizy SpA (HZY)', 'Homizy SpA (HZY)', 'Homizy SpA (HZY)', 'Homizy SpA (HZY)(English: Homizy SpA (HZY)) is a public company that is listed on Milan Stock Exchange.', 'Homizy SpA (HZY)(English: Homizy SpA (HZY)) is a public company that is listed on Milan Stock Exchange.', 'Homizy SpA (HZY)(English: Homizy SpA (HZY)) is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company.', 'Homizy SpA (HZY)(English: Homizy SpA (HZY)) is a public company that is listed on Milan Stock Exchange. They are providing products/services in Italy, and they are operated as public company.', '', '', '', '', '', '', '', '', '', '', '', '', 'Italy', 'Italy', 'Italy, Europe', 'Italy, Europe', 'N/A', '', '', '', 'Listed', '', '', 'Director', 'Director', 'Executive Board', '', '', '', '', '', '', '', '', '', '', '', '', '', ''